In [1]:
# %load_ext autoreload
# %autoreload 2

### Authors:
- [Karan Bhagat](mailto:kb.in.triangulum@gmail.com) - Developer
- [Christine Task](mailto:christine.task@knexusresearch.com) - Project technical lead
- [Dhruv Kapur](mailto:dhruvka@umich.edu) - Contributor

## NIST Collaborative Research Cycle (CRC)
### Example Notebook 3: Race Distribution (difference in count of records of each race in deid dataset and target dataset).

One concern with data deidentification is the difficulty of maintaining accurate data distributions on data with diverse subpopulations. Privacy algorithms tend to reduce the visibility of outliers-- either intentionally, through Statistical Disclosure Limitation suppression schemes such as k-anonymity, or indirectly, through additive noise that has a relatively bigger impact on small counts (ie, a count of 2 +/- 2 retains less information than a count of 200 +/- 2).  When a diverse subpopulation has a smaller number of individuals, and those individuals have feature combinations that differ from the majority, that group can be dispersed more thinly across the feature space and then become significantly altered or erased during deidentification.

In this notebook we show how to check in on this potential problem for our deidentified data samples, using a simple metric that compares the edit distance between the race distribution in the target data and deidentified data (using the csv data files for each). At the end of the notebook we print a data frame containing this metric and other algorithm properties that may be worthy of further exploration.

#### List of topics covered in this notebook:
0. [Setup notebook.](#nb3-section-0)
1. [Load deid datasets index file (index.csv).](#nb3-section-1)  
<!--    1.1 [List properties (columns) of each deid. dataset (each row) in the index dataframe (created from index.csv).](#section-1.1)  -->
2. [CCompute difference in densities of each race between deid and target datasets.](#nb3-section-2)  

<a id="nb3-section-0"></a>
## Setup notebook.

### Import required libraries

In [2]:
# import required libraries
from pathlib import Path  # for creating paths for the filesystem operations
import os  # for filesystem operations
import pandas as pd  # for tabular data operations
import numpy as np  # for tabular data operations
import matplotlib.pyplot as plt  # for plotting data as charts
import json

### Import local libraries present in libs package

In [3]:
# Import helper constants and functions
# libs.util has some small utility functions
# such has read_json, list_report etc.
import libs.util as u
# libs.plot module contains some helper
# funtions to create matplotlib charts
import libs.plot as plot
# libs.strs contains some constant strings
# represeting columns/feature names in index.csv file,
# report.json file, etc.
# Some examples of such constants: 
# LIBRARY_NAME, ALOGIRTHM_NAME, TARGET_DATASETetc.
from libs.strs import *
# lib.strs provides get_colors function that can be
# use to generate N random colors.
import libs.color as color

### Change Current Working Directory (CWD) to: crc_data_and_metric_bundle

In [4]:
print('CWD Before: ', Path.cwd())
# get current working directory
cwd = Path.cwd()
# get path to the directory where is notebook is located
notebook_dir = u.notebook_path()
# Name of the parent directory of this notebook is crc_data_and_metric_bundle_1.0
new_cwd = notebook_dir.parent
# Set new current working directory path which ends in crc_data_and_metric_bundle_1.0
os.chdir(new_cwd)
print('CWD After: ', Path.cwd())

CWD Before:  /Users/karan/Documents/Knexus/NIST/SDNist/crc_acceleration_bundle_1.1/crc_data_and_metric_bundle_1.1/notebooks
CWD After:  /Users/karan/Documents/Knexus/NIST/SDNist/crc_acceleration_bundle_1.1/crc_data_and_metric_bundle_1.1


<a id="nb3-section-1"></a>
## 1. Load deid datasets index file (index.csv)
index.csv file has tabular information about the deidentified (deid) datasets created using different data-deidentification libraries and algorithms. 
#### Location of index.csv: *crc_data_and_metric_bundle_1.0/index.csv*
#### Location of this notebook: *crc_data_and_metric_bundle_1.0/notebooks/intro.ipynb*

In [5]:
CWD = Path.cwd()  # get current working directory path
# Create path to the index.csv file
INDEX_FILE_PATH = Path(CWD, 'index.csv')
# read index csv into idx_df variable
# idx stands for index
idx_df = pd.read_csv(INDEX_FILE_PATH, index_col=0).reset_index(drop=True)

Index data has a total of 298 different deid. datasets and each dataset has 23 different properties.

In [6]:
# See the shape of the index dataframe.
idx_df.shape

(458, 22)

In [7]:
# See five random rows of the index dataframe
idx_df.sample(5)

,library name,algorithm name,algorithm type,target dataset,feature set name,feature space size,features list,privacy category,privacy label detail,epsilon,...,variant label detail,research papers,data path,labels path,report path,team,submission number,submission timestamp,quasi identifiers subset,deid data id
420,tumult,DPHist,histogram,national2019,demographic-focused-except-AGEP-DEYE,1135134,"SEX, MSP, RAC1P, HOUSING_TYPE, OWN_RENT, EDU, ...",dp,"Differentially private, histogram-based synthe...",1.0,...,NaN,https://web.cs.ucdavis.edu/~franklin/ecs289/20...,deid_data/tumult_dphist/national/dphist_e_1_cf...,deid_data/tumult_dphist/national/dphist_e_1_cf...,deid_data/tumult_dphist/national/report_dphist...,CRC,NaN,5/20/2023 00:00:00,NaN,4a5ef378898955b24257566d7bcc365b00701bb3
311,smartnoise-synth,patectgan,neural net,tx2019,industry-focused,32432400,"PUMA, SEX, MSP, HISP, RAC1P, OWN_RENT, INDP_CA...",dp,Conditional tabular GAN using Private Aggregat...,5.0,...,NaN,https://openreview.net/forum?id=S1zk9iRqF7,deid_data/smartnoise_patectgan/tx/patectgan_e_...,deid_data/smartnoise_patectgan/tx/patectgan_e_...,deid_data/smartnoise_patectgan/tx/report_patec...,CRC,NaN,5/20/2023 00:00:00,NaN,95f827acc71472a99b9a733f6241c0a5ce0a69d4
299,smartnoise-synth,patectgan,neural net,national2019,simple-features,196151155200000000000,"PUMA, AGEP, SEX, MSP, HISP, RAC1P, NOC, NPF, H...",dp,Conditional tabular GAN using Private Aggregat...,1.0,...,NaN,https://openreview.net/forum?id=S1zk9iRqF7,deid_data/smartnoise_patectgan/national/patect...,deid_data/smartnoise_patectgan/national/patect...,deid_data/smartnoise_patectgan/national/report...,CRC,NaN,5/20/2023 00:00:00,NaN,d2bae376f0c8d58bb3109007ec52d84b2a9b4a55
410,synthcity,tvae,neural net,ma2019,industry-focused,27027000,"PUMA, SEX, MSP, HISP, RAC1P, OWN_RENT, INDP_CA...",non_dp,A conditional VAE network which can handle tab...,NaN,...,added a column target to the original dataset ...,https://doi.org/10.48550/arXiv.1907.00503,deid_data/sythcity_tvae/ma/tvae_industry.csv,deid_data/sythcity_tvae/ma/tvae_industry.json,deid_data/sythcity_tvae/ma/report_tvae_industr...,CRC,NaN,5/20/2023 00:00:00,NaN,040ed2f17bccfc27c53bff0586b3fc6f5fdd6971
46,rsynthpop,cart,stat model,ma2019,industry-focused,27027000,"PUMA, SEX, MSP, HISP, RAC1P, OWN_RENT, INDP_CA...",non_dp,Non-differentially private synthetic data. Pr...,NaN,...,NaN,https://doi.org/10.18637/jss.v074.i11,deid_data/rsynthpop_cart/ma/cart_industry_ma20...,deid_data/rsynthpop_cart/ma/cart_industry_ma20...,deid_data/rsynthpop_cart/ma/report_cart_indust...,CRC,NaN,5/20/2023 00:00:00,NaN,dde7ecbf0a1e39a35314e638f2fe6f55cfc68fe4


### 1.1 Veiw available features in the index dataframe.
List features(columns) of each deid. dataset (each row) in the index dataframe (created from index.csv). 

Some features inform about the method of data synthesis (features such as 'library', 'algorithm name', 'epsilon', 'variant label' etc.) and others about the location of resources (such as 'data path' path to deid dataset csv file, 
'report path' path to sdnist evaluation report of the deid dataset file etc.)

In [8]:
# Iterate over the columns of index dataframe
# and print each column name
for col in idx_df.columns:
    print(col)

library name
algorithm name
algorithm type
target dataset
feature set name
feature space size
features list
privacy category
privacy label detail
epsilon
delta
variant label
variant label detail
research papers
data path
labels path
report path
team
submission number
submission timestamp
quasi identifiers subset
deid data id


In [9]:
# Use index_feature_description function from the libs.util module.
# index_feature_description function takes just one argument which 
# should be one of the column name available in the index file. 
# Column names are printed in the last code cell for your reference.
u.index_feature_description(LIBRARY_NAME)


Index Column: 
-------------
library name


Description:
-----------
Software library used for this submission, or team name if a private codebase was used


<a id="nb3-section-2"></a>
## 2. Compute difference in densities percentage of each race between deid and target datasets.
Race density is computed by finding the fraction of people of a race in a dataset. We find race density percentage for both a deid dataset and its target dataset, and then take the absolute difference of these density percentages.
We then use the absolute difference of density percentages to create a new score called race_edit_distance_percent.

In [10]:
# Store path to the target dataset base directory which is 
# diverse_communities_data_excerpts.
TARGET_DATA_DIR = Path(CWD, 'diverse_communities_data_excerpts')
# Store path to all the three datasets: ma2019, tx2019, and national2019
MA_PATH = Path(TARGET_DATA_DIR, 'massachusetts', 'ma2019.csv')
TX_PATH = Path(TARGET_DATA_DIR, 'texas', 'tx2019.csv')
NAT_PATH = Path(TARGET_DATA_DIR, 'national', 'national2019.csv')


# Declare constants that we use as
# column names
DEID_COUNT = 'deid_count'
DEID_DENSITY_PERCENT = 'deid_density_percent'
TARGET_COUNT = 'target_count'
TARGET_DENSITY_PERCENT = 'target_density_percent'
DIFFERENCE = 'difference'
RACE_EDIT_DISTANCE_PERCENT = 'race_edit_distance_percent'


# Create a new zero filled column
idx_df[RACE_EDIT_DISTANCE_PERCENT] = np.zeros(idx_df.shape[0]).astype(int)

# Iterate over each row (deid dataset) in index dataframe.
for i, row in idx_df.iterrows():
    # Get path of deid. data csv file
    data_path = row[DATA_PATH]
    # Read the csv file into a pandas dataframe using data path.
    data_deid_df = pd.read_csv(data_path)
    # Cleaning step (removes extra leading column from csv).
    data_deid_df = data_deid_df.loc[:, ~data_deid_df.columns.str.startswith('Unnamed')]
    data_deid_df = data_deid_df.reset_index(drop=True)
    
    # Get path to the target dataset from which the
    # current deid dataset is created
    target_dataset_name = row[TARGET_DATASET]
    target_path = ''
    
    if target_dataset_name == MA2019:
        target_path = MA_PATH
    elif target_dataset_name == TX2019:
        target_path = TX_PATH
    elif target_dataset_name == NATIONAL2019:
        target_path = NAT_PATH
    
    data_target_df = pd.read_csv(target_path)
    
    if RAC1P not in data_deid_df.columns.tolist():
        continue

    # Get count of each race in deid dataset and save it as new dataframe
    deid_race_df = data_deid_df[RAC1P].value_counts().to_frame(DEID_COUNT)
    # Compute density percent of each race in the deid dataset
    deid_race_df[DEID_DENSITY_PERCENT] = deid_race_df[DEID_COUNT] / data_deid_df.shape[0] * 100
    
    # Get count of each race in target dataset and save it as new dataframe
    target_race_df = data_target_df[RAC1P].value_counts().to_frame(TARGET_COUNT)
    # Compute density percent of each race in the target dataset
    target_race_df[TARGET_DENSITY_PERCENT] = target_race_df[TARGET_COUNT] / data_target_df.shape[0] * 100
    
    # Move unique values of RAC1P to a new column named RAC1P and set type as
    # 'int'. We change type to 'int' because some deid dataset save RAC1P
    # value as floats.
    deid_race_df[RAC1P] = deid_race_df.index.astype(int)
    target_race_df[RAC1P] = target_race_df.index.astype(int)

    # Reset index after get race values
    deid_race_df = deid_race_df.reset_index(drop=True)
    target_race_df = target_race_df.reset_index(drop=True)
    
    # Sort deid and target dataset by race
    deid_race_df = deid_race_df.sort_values(by=RAC1P)

    target_race_df = target_race_df.sort_values(by=RAC1P)

    # Merge both dataset on RAC1P and do outer merge to take RAC1P values from
    # both deid and target dataset
    merged_df = target_race_df.merge(deid_race_df, on=RAC1P, how='outer')
    
    # Since few races can be missing in the deid dataset, those
    # will have nan values for deid_counts and deid_density_percent
    # columns after the merge operation
    merged_df = merged_df.fillna(0)
    # Get absolute difference of density percentages between target and deid dataset
    merged_df[DIFFERENCE] = abs(merged_df[DEID_DENSITY_PERCENT] - merged_df[TARGET_DENSITY_PERCENT])
        
    # Calculate edit distance between density percentages of races in 
    # target data and count of races in deid data
    edit_distance = sum(merged_df[DIFFERENCE].values)

    # Save value race values difference in
    # index dataframe.
    idx_df.loc[i, RACE_EDIT_DISTANCE_PERCENT] = round(edit_distance, 2)

### 2.1 Worst performing 75 samples.

In [11]:
# See the random 100 records from index dataframe and
# the new column race_edit_distance which we just added to the
# index dataframe

# Set pandas display max rows option to display more
# dataframe rows in the cell output
pd.set_option('display.max_rows', 151)
# Name of the columns to display in the selected data samples
columns_to_show = [LIBRARY_NAME, ALGORITHM_NAME, ALGORITHM_TYPE, TARGET_DATASET,
                   PRIVACY_CATEGORY, EPSILON, FEATURE_SET_NAME, RACE_EDIT_DISTANCE_PERCENT]

# Sort the samples by race_edit_distance
idx_df = idx_df.sort_values(by=RACE_EDIT_DISTANCE_PERCENT, ascending=False)
# Show 75  worst performing samples in the index dataframe
idx_sample_df = idx_df[columns_to_show].sample(75)
# Show samples dataframe
idx_sample_df

,library name,algorithm name,algorithm type,target dataset,privacy category,epsilon,feature set name,race_edit_distance_percent
98,sdcmicro,pram,sdc,ma2019,sdc,NaN,demographic-focused,0.76
114,sdv,ctgan,neural net,national2019,non_dp,NaN,all-features,16.84
72,rsynthpop,ipf,query matching,national2019,dp,2.0,demographic-focused,2.67
205,smartnoise-synth,mst,stat model,ma2019,dp,1.0,industry-focused,2.44
155,smartnoise-synth,aim,stat model,ma2019,dp,10.0,simple-features,0.81
40,aindo-synth,aindo-synth,neural net,national2019,non_dp,NaN,all-features,1.66
0,Anonos Data Embassy SDK,Anonos Data Embassy SDK,neural net,ma2019,non_dp,NaN,all-features,0.79
409,synthcity,tvae,neural net,ma2019,non_dp,NaN,family-focused,8.07
229,smartnoise-synth,mst,stat model,tx2019,dp,10.0,demographic-focused-except-DEYE,0.30
384,synthcity,pategan,neural net,tx2019,dp,1.0,demographic-focused,13.95


In [12]:
### 2.1 Best performing 75 samples.

In [13]:
# See the random 100 records from index dataframe and
# the new column race_edit_distance which we just added to the
# index dataframe

# Set pandas display max rows option to display more
# dataframe rows in the cell output
pd.set_option('display.max_rows', 151)
# Name of the columns to display in the selected data samples
columns_to_show = [LIBRARY_NAME, ALGORITHM_NAME, ALGORITHM_TYPE, TARGET_DATASET,
                   PRIVACY_CATEGORY, EPSILON, FEATURE_SET_NAME, RACE_EDIT_DISTANCE_PERCENT]

# Sort the samples by race_edit_distance
idx_df = idx_df.sort_values(by=RACE_EDIT_DISTANCE_PERCENT, ascending=True)
# Show 75 best performing samples in the index dataframe
idx_sample_df = idx_df[columns_to_show].head(75)
# Show samples dataframe
idx_sample_df

,library name,algorithm name,algorithm type,target dataset,privacy category,epsilon,feature set name,race_edit_distance_percent
449,ydata-sdk,YData Fabric Synthesizers,neural net,tx2019,non_dp,NaN,custom-features-3,0.00
428,ydata-sdk,YData Fabric Synthesizers,neural net,ma2019,non_dp,NaN,custom-features-3,0.00
429,ydata-sdk,YData Fabric Synthesizers,neural net,ma2019,non_dp,NaN,custom-features-5,0.00
440,ydata-sdk,YData Fabric Synthesizers,neural net,national2019,non_dp,NaN,custom-features-5,0.00
58,rsynthpop,catall,histogram,national2019,dp,100.0,demographic-focused,0.00
450,ydata-sdk,YData Fabric Synthesizers,neural net,tx2019,non_dp,NaN,custom-features-5,0.00
439,ydata-sdk,YData Fabric Synthesizers,neural net,national2019,non_dp,NaN,custom-features-3,0.00
35,UTDallas-AIFairness,smote,geometric,national2019,non_dp,NaN,all-features,0.00
38,UTDallas-AIFairness,smote,geometric,tx2019,non_dp,NaN,all-features,0.00
210,smartnoise-synth,mst,stat model,national2019,dp,10.0,demographic-focused,0.05
